In [2]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

])

In [6]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

100%|██████████| 170498071/170498071 [00:02<00:00, 76292029.14it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
image, label = train_data[0]

In [8]:
image.size()

torch.Size([3, 32, 32])

In [9]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'frog', 'horse', 'ship', 'truck']

In [14]:
class NeuralNet(nn.Module):

  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(3, 12, 5) # (32 - 5) / 1 + 1 (12, 28, 28)
    self.pool = nn.MaxPool2d(2, 2) # (12, 14, 14)
    self.conv2 = nn.Conv2d(12, 24, 5)
    self.fc1 = nn.Linear(24 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [15]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum =0.9)

In [19]:
for epoch in range(30):
  print(f'Training epoch {epoch}...')

  running_loss = 0.0

  for i, data in enumerate(train_loader):
    inputs, labels = data

    optimizer.zero_grad()

    outputs = net(inputs)

    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
  print(f'loss: {running_loss/len(train_loader):.4f}')

Training epoch 0...
loss: 1.7524
Training epoch 1...
loss: 1.5257
Training epoch 2...
loss: 1.4010
Training epoch 3...
loss: 1.3101
Training epoch 4...
loss: 1.2238
Training epoch 5...
loss: 1.1488
Training epoch 6...
loss: 1.0884
Training epoch 7...
loss: 1.0319
Training epoch 8...
loss: 0.9856
Training epoch 9...
loss: 0.9430
Training epoch 10...
loss: 0.9082
Training epoch 11...
loss: 0.8742
Training epoch 12...
loss: 0.8429
Training epoch 13...
loss: 0.8075
Training epoch 14...
loss: 0.7807
Training epoch 15...
loss: 0.7521
Training epoch 16...
loss: 0.7241
Training epoch 17...
loss: 0.7012
Training epoch 18...
loss: 0.6795
Training epoch 19...
loss: 0.6539
Training epoch 20...
loss: 0.6290
Training epoch 21...
loss: 0.6079
Training epoch 22...
loss: 0.5869
Training epoch 23...
loss: 0.5647
Training epoch 24...
loss: 0.5430
Training epoch 25...
loss: 0.5218
Training epoch 26...
loss: 0.5043
Training epoch 27...
loss: 0.4802
Training epoch 28...
loss: 0.4628
Training epoch 29...
los

In [21]:
torch.save(net.state_dict(), 'trained_net.pth')

In [22]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<ipython-input-22-c299e2fffacc>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [23]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
  for data in test_loader:
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()


accuracy = 100 *correct / total

print(f'Accuracy {accuracy}%')

Accuracy 10.0%


In [ ]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0,5))
])

def load_image(image_path):
  image = Image.open(image_path)
  image = new_transform(image)
  image = image.unsqueeze(0)
  return image


image_paths = ['example1.jpg', 'example2.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
  for image in images:
    output = net(image)
    _, predicted = torch.max(outputs, 1)
    print(f'Prediction: {class_names[predicted.item()]}')